# Demonstrate using *scikit-learn* to perform supervised clustering
## With some advanced ML concepts
This was originally coded in 2016, as part of a stand-alone python module

In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import pairwise
from sklearn import svm
from sklearn import cross_validation
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression

/home/ahowe42/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ahowe42/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# get the data
iris = load_iris()
data = iris.data
labels = iris.target
(n,p) = data.shape
print('Observations = %d, Features = %d'%(n,p))

Observations = 150, Features = 4
